In [1]:
import logging

from time import time, sleep
from multiprocessing import Process, Manager
logging.basicConfig()

from kazoo.client import KazooClient

MEAL_TIME_SEC = 1
WAITING_TIME_SEC = 0.1

In [7]:
class Philosopher(Process):
    def __init__(self, task_name: str, id: int, fork_path: str, eat_seconds: int = 15, max_id: int = 5):
        super().__init__()
        self.root = task_name
        self.fork = fork_path
        self.id = id
        self.left_fork_id = id
        self.right_fork_id = id + 1 if id + 1 < max_id else 0
        self.eat_seconds = eat_seconds
        self.partner_id_left = id - 1 if id - 1 >= 0 else max_id-1
        self.partner_id_right = id + 1 if id + 1 < max_id else 0
        
    def run(self):
        zk = KazooClient()
        zk.start()

        table_lock = zk.Lock(f'{self.root}/table', self.id)
        left_fork = zk.Lock(f'{self.root}/{self.fork}/{self.left_fork_id}', self.id)
        right_fork = zk.Lock(f'{self.root}/{self.fork}/{self.right_fork_id}', self.id)

        start = time()
        while time() - start < self.eat_seconds:
            with table_lock:
                if len(left_fork.contenders()) == 0 and len(right_fork.contenders()) == 0 \
                        and counters[self.partner_id_right] >= counters[self.id] \
                        and counters[self.partner_id_left] >= counters[self.id]:
                    left_fork.acquire()
                    right_fork.acquire()
            if left_fork.is_acquired and right_fork.is_acquired:
                print(f'Philosopher {self.id}: Im eating')
                counters[self.id] += 1
                sleep(MEAL_TIME_SEC)
                left_fork.release()
                right_fork.release() 
            else:
                print(f'Philosopher {self.id}: Im thinking')
            sleep(WAITING_TIME_SEC)
            
        print(f'Philosopher with id={self.id} eat counter={counters[self.id]}!')
        zk.stop()
        zk.close()

In [8]:
master_zk = KazooClient()
master_zk.start()
if master_zk.exists('/task1'):
    master_zk.delete('/task1', recursive=True)

master_zk.create('/task1')
master_zk.create('/task1/table')
master_zk.create('/task1/forks')
master_zk.create('/task1/forks/1')
master_zk.create('/task1/forks/2')
master_zk.create('/task1/forks/3')
master_zk.create('/task1/forks/4')
master_zk.create('/task1/forks/5')

root = '/task1'
fork_path = 'forks'
seconds_eat = 30

counters = Manager().list()
p_list = list()
for i in range(0, 5):
    p = Philosopher(root, i, fork_path, seconds_eat)
    counters.append(0)
    p_list.append(p)
    
for p in p_list: 
    p.start()

Philosopher 0: Im eating
Philosopher 1: Im thinking
Philosopher 2: Im eating
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im thinking
Philosopher 1: Im thinking
Philosopher 3: Im thinking
Philosopher 4: Im eating
Philosopher 1: Im eating
Philosopher 0: Im thinking
Philosopher 3: Im thinking
Philosopher 2: Im thinking
Philosopher 0: Im thinking
Philosoph